# CVRP Solver with Density-Based Clustering

## Project Overview

This notebook implements an advanced solver for the Capacitated Vehicle Routing Problem (CVRP) using a hybrid approach that combines:

- **KMeans Clustering**: Initial spatial partitioning of delivery points
- **Kernel Density Estimation (KDE)**: Identifying high-density regions
- **DBSCAN**: Sub-clustering within each partition for fine-grained route optimization
- **OR-Tools**: TSP solving for route sequencing

## Problem Context

The CVRP is a classic optimization problem where:
- Multiple vehicles with limited capacity must serve a set of delivery points
- Each delivery has a specific size/demand
- The goal is to minimize total travel distance while respecting vehicle capacity constraints

## Approach

1. **Training Phase**: Learn density patterns from historical delivery data
2. **Clustering Phase**: Partition new deliveries into spatial regions
3. **Routing Phase**: Optimize routes within each partition
4. **Solution Phase**: Combine individual routes into a complete solution

## Data Source

Using LoggiBUD dataset for CVRP instances with real-world delivery locations.

In [2]:
# Standard library imports
import re
import sys
import uuid
import warnings
from dataclasses import dataclass, field
from io import BytesIO
from itertools import compress
from math import sqrt
from pathlib import Path
from typing import Dict, List, Optional

# Numerical and scientific computing
import numpy as np
import pandas as pd
import scipy.stats as st
import scipy as sp
from math import sqrt

# Machine learning and clustering
from sklearn.cluster import DBSCAN, KMeans

# Optimization
from ortools.constraint_solver import pywrapcp

# Visualization
import folium
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import plotly.express as px

# LoggiBUD specific imports
sys.path.append(str(Path().joinpath('..')))
from loggibud.v1.distances import OSRMConfig, calculate_distance_matrix_m
from loggibud.v1.eval.task1 import evaluate_solution
from loggibud.v1.plotting.plot_instance import plot_cvrp_instance
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution
from loggibud.v1.types import (
    CVRPInstance,
    CVRPSolution,
    CVRPSolutionVehicle,
    Delivery,
    Point,
)

# Configure warnings
warnings.filterwarnings('ignore')

## OSRM Configuration

### Overview

OSRM (Open Source Routing Machine) is used to calculate real-world road distances between delivery points. This provides more accurate routing than simple Euclidean distances. The OSRM client is configured to connect to the local server.

### Configuration

1. **Docker** must be installed and running

2. **OSRM data file** for Brazil (already available in `osrm/brazil-201110.osrm`)

### Starting the OSRM Server 

Run this command in your terminal from the project root:

```
docker run -t -i -p 5001:5000 -v "${PWD}:/data" osrm/osrm-backend:v5.24.0 osrm-routed --algorithm ch /data/osrm/brazil-201110.osrm --max-table-size 10000
```
- `-p 5001:5000`: Maps container port 5000 to host port 5001
- `--algorithm ch`: Uses Contraction Hierarchies for fast routing
- `--max-table-size 10000`: Allows distance matrix calculations for up to 10,000 points

In [3]:
# Configure OSRM client to connect to local server on port 5001
osrm_config = OSRMConfig(host="http://localhost:5001")

# Display configuration
osrm_config

OSRMConfig(host='http://localhost:5001', timeout_s=6000)

## Density Estimation

This section implements Kernel Density Estimation (KDE) to analyze the spatial distribution of delivery points. KDE helps identify areas with high delivery concentration, which is crucial for effective route clustering.

The density estimation workflow consists of:
1. **Computing KDE**: Calculate density estimates using Gaussian kernels over a grid
2. **Visualizing Contours**: Plot density contours to visualize delivery concentration areas
3. **Classifying Points**: Determine which density regions each delivery point belongs to

These functions will be used by the KMeansSolver to improve cluster formation based on delivery density patterns.

### Compute Density Contours

This function performs Kernel Density Estimation (KDE) on a set of 2D points to create a smooth density surface. It uses a Gaussian kernel to estimate the probability density function across a grid of coordinates.

**Purpose**: Create density contours that represent areas of high/low delivery concentration.

**Returns**: Grid coordinates (x, y, xx, yy) and density values (f) for contour plotting.

In [4]:
def set_contours(points):
    """
    Compute kernel density estimation contours for a set of 2D points.
    
    Args:
        points: List of [x, y] coordinate pairs
        
    Returns:
        tuple: (x, y, xx, yy, f) where:
            - x, y: Original point coordinates
            - xx, yy: Meshgrid coordinates for the density surface
            - f: Density values at each grid point
    """
    # Extract x and y coordinates from the points list
    x = [i[0] for i in points]
    y = [i[1] for i in points]

    # Find the minimum and maximum coordinates to define the grid boundaries
    xmin, xmax = np.min(x), np.max(x)
    ymin, ymax = np.min(y), np.max(y)

    # Create a 100x100 meshgrid spanning the coordinate range
    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    
    # Flatten the meshgrid into a 2D array of positions
    positions = np.vstack([xx.ravel(), yy.ravel()])
    
    # Stack the original coordinates for KDE input
    values = np.vstack([x, y])
    
    # Compute Gaussian kernel density estimation
    kernel = st.gaussian_kde(values)
    
    # Evaluate the KDE at each grid position and reshape to match the meshgrid
    f = np.reshape(kernel(positions).T, xx.shape)
    
    # Return all components needed for plotting and analysis
    return x, y, xx, yy, f

### Visualize Density Contours

This function creates a visual representation of the density estimation by plotting filled contours. It uses matplotlib's `contourf` to show regions of different density levels.

**Purpose**: Generate contour plots to visualize delivery concentration areas and optionally overlay the original points.

**Returns**: Contour collections (excluding background) and the contour plot object for further processing.

In [5]:
def plot_contours(xx, yy, f, x=[], y=[]):
    """
    Plot density contours and optionally overlay original data points.
    
    Args:
        xx, yy: Meshgrid coordinates from set_contours()
        f: Density values at each grid point
        x, y: Optional original point coordinates to overlay on the plot
        
    Returns:
        tuple: (contours, contour_plot) where:
            - contours: List of contour collection objects (excluding background)
            - contour_plot: The matplotlib contourf plot object
    """
    # Create a new matplotlib figure
    fig = plt.figure()
    
    # Switch to AGG backend for non-interactive plotting
    plt.switch_backend('AGG')
    
    # Set the figure size to 10x5 inches
    plt.figure(figsize=(10, 5))
    
    # Create filled contour plot using the 'bone' colormap
    contour_plot = plt.contourf(xx, yy, f, cmap='bone')

    # If original points are provided, overlay them as red dots
    if len(x) > 0 and len(y) > 0:
        plt.plot(x, y, ".r", markersize=6)

    # Adjust subplot parameters for a tight layout
    plt.tight_layout()
    
    # Get the current axes object
    ax = plt.gca()
    
    # Extract contour collections from axes (filter for objects with paths)
    contours = [col for col in ax.collections if hasattr(col, 'get_paths')]
    
    # Remove the first contour level (background) if multiple levels exist
    if len(contours) > 0:
        contours = contours[1:]
    
    # Return contour collections and plot object
    return contours, contour_plot

### Classify Points by Density Level

This function determines which density contour levels contain each delivery point. It counts how many contour regions each point falls within, providing a measure of the point's density level.

**Purpose**: Assign density scores to delivery points based on their location within contour levels. Points in multiple nested contours have higher density scores.

**Returns**: Array of integers where each value represents the number of contour levels containing that point.

In [6]:
def find_contours(x, y, contours):
    """
    Determine which density contour levels contain each point.
    
    Args:
        x, y: Arrays of point coordinates
        contours: List of matplotlib contour collection objects
        
    Returns:
        numpy.ndarray: Array where each value is the count of contour levels 
                      containing the corresponding point. Higher values indicate
                      points in denser regions.
    """
    # Initialize list to store containment results for each contour level
    inside = []
    
    # For each contour collection, check which points it contains
    for c in contours:
        # Verify the contour has at least one path
        if len(c.get_paths()) > 0:
            # Get the first path from the collection
            p = c.get_paths()[0]
            
            # Check if each (x,y) point is inside this contour path
            inside.append(p.contains_points(list(zip(*(x, y)))))

    # Calculate the density level for each point
    if len(inside) > 0:
        # Sum along axis 0 to count how many contour levels contain each point
        result = np.sum(inside, axis=0)
        return result
    else:
        # If no contours exist, return zeros for all points
        return np.zeros(len(x), dtype=int)

## Routing Functions

This section contains utility functions for route construction and optimization. These functions support the main solver by:

1. **Cluster Management**: Adding cluster centroids as map markers
2. **Sub-route Organization**: Preparing and aligning sub-routes based on density clusters
3. **Distance Calculations**: Computing Euclidean distances and reordering deliveries
4. **Helper Functions**: Extracting coordinates and computing vehicle volumes

These functions work together to transform clustered delivery points into optimized vehicle routes.

### Add Cluster Centroids to Map Markers

This function converts KMeans cluster centroids into map marker objects for visualization. Each cluster center becomes a black pin on the map.

**Purpose**: Enable visualization of cluster centers on the delivery map, helping to understand the spatial distribution of route assignments.

**Returns**: List of marker dictionaries with latitude, longitude, cluster ID, color, type, and tooltip information.

In [7]:
def add_clusters_to_markers(model):
    """
    Convert KMeans cluster centroids into map marker objects.
    
    Args:
        model: A fitted KMeans model with cluster_centers_ attribute
        
    Returns:
        list: List of marker dictionaries, each containing:
            - lat, lng: Centroid coordinates
            - cluster: Cluster index
            - color: Marker color (black for centroids)
            - type: List of marker types ['cluster', 'pin']
            - tooltip: Display text for the marker
    """
    # Initialize empty list to store marker dictionaries
    markers = []
    
    # Iterate through each cluster centroid
    for i, center in enumerate(model.cluster_centers_):
        # Create a marker dictionary for this centroid
        m = dict()
        m['lat'] = center[0]       # Latitude coordinate of the centroid
        m['lng'] = center[1]       # Longitude coordinate of the centroid
        m['cluster'] = i           # Cluster index number
        m['color'] = 'black'       # Color for cluster centroids
        m['type'] = ['cluster', 'pin']  # Marker type identifiers
        m['tooltip'] = 'Cluster {}'.format(i)  # Hover text showing cluster number
        
        # Add this marker to the list
        markers.append(m)
        
    # Return the complete list of cluster markers
    return markers

### Prepare Sub-routes Structure

This function initializes a dictionary structure to organize deliveries into sub-routes based on density clustering. It creates keys for all combinations of cluster IDs, contour levels, and DBSCAN labels.

**Purpose**: Create a hierarchical structure to group deliveries by cluster, density level, and sub-cluster, enabling fine-grained route organization.

**Returns**: Tuple of (subroutes dictionary, list of unique labels) for the specified cluster.

In [8]:
def prepare_sub_routes(db_predictions, cluster):
    """
    Initialize sub-route dictionary structure for hierarchical route organization.
    
    Args:
        db_predictions: DBSCAN cluster labels for points
        cluster: Current KMeans cluster ID
        
    Returns:
        tuple: (subroutes dict, labels list) where:
            - subroutes: Dictionary with keys like "k_0_contour_-1_label_0"
            - labels: List of unique DBSCAN labels including -1 and -99
    """
    # Initialize empty dictionary for sub-routes
    subroutes = dict()
    
    # Get unique DBSCAN labels from predictions
    labels = list(np.unique(db_predictions))
    
    # Ensure label -1 (noise points) is included in the list
    if -1 not in labels:
        labels = labels + [-1]
    
    # Ensure label -99 (special case) is included in the list
    if -99 not in labels:
        labels = labels + [-99]
    
    # Create sub-route keys for each combination of contour level (-1, 0) and label
    for j in [-1, 0]:
        for l in labels:
            # Build key: cluster ID + contour level + DBSCAN label
            subroutes["k_" + str(cluster) + '_contour_' + str(j) + "_label_" + str(l)] = []

    # Return the initialized structure and label list
    return subroutes, labels

### Align Sub-routes by Centroid Order

This function reorders sub-routes based on the sequence of ordered centroids within each cluster. It ensures sub-routes are processed in a logical spatial order.

**Purpose**: Organize sub-routes to match the spatial ordering of centroids, enabling more efficient sequential route construction.

**Returns**: Ordered dictionary mapping positions to sub-route dictionaries, sorted by centroid sequence.

In [9]:
def align_subroutes(density_info):
    """
    Reorder sub-routes based on the spatial sequence of ordered centroids.
    
    Args:
        density_info: Dictionary containing cluster information including:
            - ordered_centroids: List of centroids in spatial order
            - subroutes_cluster: Dictionary of sub-routes by cluster/label
        
    Returns:
        dict: Ordered dictionary mapping positions to sub-route dictionaries,
              sorted to match centroid ordering for efficient route construction
    """
    # Initialize ordered dictionary and position counter
    ordered_subroutes = dict()
    latest_pos = 0
    
    # Iterate through each cluster in density_info
    for k, v in density_info.items():
    
        # Process each centroid in the ordered sequence
        for pos, centroid in enumerate(v['ordered_centroids']):
            
            # Extract cluster label from centroid ID using regex pattern
            r = re.search(r"(cluster)_(\d+)_(\d+)", centroid.id)
            label = list(r.groups())[2]
            
            # Find matching sub-route by label
            for k2, v2 in v['subroutes_cluster'].items():
    
                # Extract label from sub-route key using regex
                r = re.search(r"(k)_(-?\d+)_(contour)_(-?\d+)_(label)_(-?\d+)", k2)
                label_ = list(r.groups())[5]
                
                # If labels match, add to ordered dictionary
                if label == label_:
                    ordered_subroutes[latest_pos] = {k2: v2}
                    latest_pos += 1
                    break
        
        # Add any remaining sub-routes that weren't matched by centroids
        for k3, v3 in v['subroutes_cluster'].items():
            # Check if this sub-route key is not already in ordered dictionary
            if k2 not in pd.DataFrame(ordered_subroutes, columns=['pos', 'k', 'v'])[['k']].values:
    
                ordered_subroutes[latest_pos] = {k3: v3}
                latest_pos += 1

    # Sort the dictionary by position keys to ensure proper ordering
    ordered_subroutes = dict(sorted(ordered_subroutes.items()))
    
    # Return the fully ordered sub-routes dictionary
    return ordered_subroutes

### Calculate Euclidean Distance

This function computes the straight-line (Euclidean) distance between two geographic points using their latitude and longitude coordinates.

**Purpose**: Provide a fast distance metric for greedy nearest-neighbor routing heuristics.

**Returns**: Float representing the Euclidean distance between the two points.

In [10]:
def euclidean_distance(p1: Point, p2: Point) -> float:
    """
    Compute the Euclidean distance between two geographic points.
    
    Args:
        p1: First point with lat and lng attributes
        p2: Second point with lat and lng attributes
        
    Returns:
        float: Straight-line distance between the two points
    """
    # Calculate distance using Pythagorean theorem: sqrt((x2-x1)^2 + (y2-y1)^2)
    return sqrt((p1.lat - p2.lat) ** 2 + (p1.lng - p2.lng) ** 2)

### Reorder Deliveries by Nearest Neighbor

This function implements a greedy nearest-neighbor heuristic to reorder deliveries. Starting from the origin, it repeatedly selects the closest unvisited delivery until all are sequenced.

**Purpose**: Create a simple but effective delivery sequence that minimizes cumulative travel distance.

**Returns**: Reordered list of deliveries starting with the origin, followed by deliveries in nearest-neighbor order.

In [11]:
def reorder_deliveries_by_distance(deliveries: List[Delivery]) -> List[Delivery]:
    """
    Reorder deliveries using a greedy nearest-neighbor heuristic.
    
    Args:
        deliveries: List of Delivery objects including one with id='origin'
        
    Returns:
        List[Delivery]: Reordered deliveries starting from origin, followed by
                       nearest-neighbor sequence
    """
    # Find and extract the origin point from the delivery list
    origin = next(d for d in deliveries if d.id == 'origin')
    
    # Create list of remaining deliveries (excluding origin)
    remaining = [d for d in deliveries if d.id != 'origin']
    
    # Initialize ordered list with the origin
    ordered = [origin]
    
    # Set current position to the origin
    current = origin

    # Continue until all deliveries are sequenced
    while remaining:
        # Find the delivery closest to the current position
        next_delivery = min(remaining, key=lambda d: euclidean_distance(current.point, d.point))
        
        # Add the closest delivery to the ordered sequence
        ordered.append(next_delivery)
        
        # Remove the selected delivery from remaining list
        remaining.remove(next_delivery)
        
        # Update current position to the newly added delivery
        current = next_delivery

    # Return the complete ordered delivery sequence
    return ordered

### Extract Delivery Coordinates

This function extracts latitude and longitude coordinates from all deliveries in an instance, converting them into a list of tuples.

**Purpose**: Transform delivery objects into coordinate pairs for clustering and spatial analysis algorithms.

**Returns**: List of (latitude, longitude) tuples for each delivery in the instance.

In [12]:
def _get_delivery_coordinates(instance):
    """
    Extract coordinate pairs from all deliveries in an instance.
    
    Args:
        instance: CVRPInstance object containing a list of deliveries
        
    Returns:
        list: List of (latitude, longitude) tuples for each delivery
    """
    # Initialize empty list for coordinate pairs
    points = []
    
    # Iterate through each delivery in the instance
    for delivery in instance.deliveries:
        # Extract lat/lng coordinates and append as a tuple
        points.append((delivery.point.lat, delivery.point.lng))

    # Return the list of coordinate pairs
    return points

### Compute Vehicle Volume

This function calculates the total volume occupied by a set of deliveries, used to verify capacity constraints.

**Purpose**: Sum the size of all deliveries to ensure they fit within vehicle capacity limits.

**Returns**: Float representing the total volume of all deliveries in the list.

In [13]:
def _compute_vehicle_volume(deliveries):
    """
    Calculate the total volume occupied by a list of deliveries.
    
    Args:
        deliveries: List of Delivery objects, each with a size attribute
        
    Returns:
        float: Sum of all delivery sizes (total volume)
    """
    # Initialize volume counter
    volume = 0
    
    # Iterate through each delivery and accumulate its size
    for delivery in deliveries:
        volume += delivery.size
    
    # Return the total volume
    return volume

### Solve TSP with OR-Tools

This function solves the Traveling Salesman Problem (TSP) using Google OR-Tools' routing optimization library. Given a distance matrix, it finds the optimal order to visit all points, starting and ending at the origin (depot).

**Purpose**: Optimize the sequence of deliveries within a route to minimize total travel distance.

**Returns**: Tuple of (route sequence, total distance) where the route is a list of node indices representing the optimal visit order.

In [14]:
def solve_tsp_ortools(distance_matrix):
    """
    Solve the Traveling Salesman Problem using Google OR-Tools.
    
    Args:
        distance_matrix: 2D array of distances between all points, where
                        distance_matrix[i][j] is the distance from point i to j
        
    Returns:
        tuple: (route, total_distance) where:
            - route: List of node indices in optimal visit order [0, 3, 1, 2, ..., 0]
            - total_distance: Total distance of the optimal route
    """
    # Convert distance matrix to integers to avoid overflow errors in OR-Tools
    # OR-Tools C++ backend expects integer values
    distance_matrix = np.array(distance_matrix, dtype=np.int64)
    
    # Get the number of nodes/points in the problem
    n = distance_matrix.shape[0]
    
    # Set number of vehicles (TSP uses only one vehicle)
    num_vehicles = 1
    
    # Set the depot node (starting and ending point, typically the origin)
    depot_node = 0
    
    # Create the routing index manager to handle node-to-index conversions
    manager = pywrapcp.RoutingIndexManager(n, num_vehicles, depot_node)
    
    # Create the routing model that will solve the TSP
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(i, j):
        """
        Callback function to provide distance between two nodes.
        
        Args:
            i, j: Internal OR-Tools indices (not node indices)
            
        Returns:
            int: Distance between nodes corresponding to indices i and j
        """
        # Convert OR-Tools internal indices to actual node indices
        ni = manager.IndexToNode(i)
        nj = manager.IndexToNode(j)
        
        # Return the distance as an integer
        return int(distance_matrix[ni, nj])

    # Register the distance callback with the routing model
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    
    # Set the cost of traveling between nodes for all vehicles
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Configure search parameters with default settings
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    
    # Solve the TSP problem
    solution = routing.SolveWithParameters(search_parameters)

    # Extract the optimal route from the solution
    route = []
    
    # Get the starting index for vehicle 0
    index = routing.Start(0)
    
    # Convert to node index and add to route
    node = manager.IndexToNode(index)
    route.append(node)

    # Follow the route until we reach the end
    while not routing.IsEnd(index):
        # Get the next index in the route
        index = solution.Value(routing.NextVar(index))
        
        # Convert to node index and add to route
        node = manager.IndexToNode(index)
        route.append(node)

    # Return the complete route and the total distance
    return route, solution.ObjectiveValue()

### DBSCAN Prediction for New Points

This function predicts which DBSCAN cluster a new delivery point belongs to. It uses a combination of:
1. Direct assignment if the point is close to existing core samples
2. Nearest centroid matching using OSRM road distances if not near core samples

**Purpose**: Assign new delivery points to existing density-based sub-clusters for consistent route grouping.

**Returns**: Array of cluster labels for the input points, matching the DBSCAN labeling scheme.

In [15]:
def dbscan_predict(density_info, X):
    """
    Predict DBSCAN cluster labels for new data points.
    
    This function assigns new points to existing DBSCAN clusters using:
    1. Direct assignment if point is within epsilon of core samples
    2. Nearest centroid matching via OSRM distances otherwise
    
    Args:
        density_info: Dictionary containing:
            - 'db': Fitted DBSCAN model with components_ and labels_
            - 'ordered_centroids': List of cluster centroids as Delivery objects
        X: Array of shape (n_samples, 2) with [lat, lng] coordinates
        
    Returns:
        numpy.ndarray: Array of cluster labels for each point in X
    """
    # Extract the fitted DBSCAN model from density information
    model = density_info['db']
    
    # Get the ordered centroids for this cluster
    ordered_centroids = density_info['ordered_centroids']

    # Get the number of samples to predict
    nr_samples = X.shape[0]

    # Initialize all predictions to -1 (noise/unassigned)
    y_new = np.ones(shape=nr_samples, dtype=int) * -1

    # Process each sample individually
    for i in range(nr_samples):
        # Calculate the difference between current point and all core samples
        # Uses NumPy broadcasting to compute element-wise differences
        diff = model.components_ - X[i, :]

        # Calculate Euclidean distance to each core sample
        dist = np.linalg.norm(diff, axis=1)

        # Initialize index for shortest distance
        shortest_dist_idx = []
        
        # Check if there are any core samples to compare against
        if len(dist) > 0:
            # Find the index of the nearest core sample
            shortest_dist_idx = np.argmin(dist)

        # If point is within epsilon distance of a core sample, assign its label
        if len(dist) > 0 and dist[shortest_dist_idx] < model.eps:
            y_new[i] = model.labels_[model.core_sample_indices_[shortest_dist_idx]]

        else:
            # Point is not near any core sample, use centroid matching instead
            
            # Create list of points starting with the current point
            ordered_centroids_points = []
            ordered_centroids_points.append(Point(lat=X[i][0], lng=X[i][1]))
            
            # Add all centroid points to the list
            for delivery in ordered_centroids:
                ordered_centroids_points.append(delivery.point)

            # Calculate OSRM road distances from current point to all centroids
            distances = calculate_distance_matrix_m(ordered_centroids_points)

            # Handle edge case where distances is returned as a single integer
            if isinstance(distances, int):
                print("Warning: Single distance value returned")
                print(ordered_centroids)
                distances = [distances]
                print(distances)

            # Find the nearest centroid (skip index 0 which is the point itself)
            shortest_cluster = np.argmin(distances[0][1:])
            
            # Get the centroid object corresponding to the shortest distance
            shortest_cluster = ordered_centroids[shortest_cluster]
            
            # Extract the cluster label from the centroid ID using regex
            # Expected format: "cluster_X_Y" where Y is the label
            r = re.search(r"(cluster)_(\d+)_(\d+)", shortest_cluster.id)
            new_label = list(r.groups())[2]
            
            # Assign the extracted label to this point
            y_new[i] = int(new_label)

    # Return the array of predicted labels
    return y_new

## KMeansSolver: Hybrid Density-Based Clustering for CVRP

This section implements the core solver that combines multiple clustering techniques to solve CVRP instances. The solver represents the main contribution of this research, using a hierarchical approach to partition deliveries based on spatial density patterns.

### Research Contribution: Multi-Level Density-Aware Clustering

The KMeansSolver introduces a novel three-stage clustering hierarchy:

1. **Macro-level Partitioning (KMeans)**: Divides the geographic space into broad regions, providing coarse-grained spatial separation

2. **Density Estimation (KDE)**: Within each macro-region, Kernel Density Estimation identifies areas of high delivery concentration. This is crucial for understanding spatial distribution patterns and separating dense urban cores from sparse peripheral areas

3. **Micro-level Clustering (DBSCAN)**: Applies density-based clustering to points outside high-density contours, creating fine-grained sub-clusters that respect natural delivery groupings

### Key Innovation: Contour-Based Density Thresholding

Points falling within high-density contours (≥4 nested levels) are treated separately from points in sparse regions. This distinction allows the solver to:
- Apply different routing strategies based on density
- Prevent over-fragmentation in dense areas
- Maintain spatial coherence in sparse regions

### Solver Workflow

1. **Training Phase** (`pretrain`): Learn density patterns from historical instances
2. **Routing Phase** (`route`): Assign new deliveries to appropriate clusters/sub-clusters
3. **Finalization Phase** (`finish`): Consolidate sub-routes into complete vehicle routes

### KMeansSolver Class Definition

The solver is implemented as a Python dataclass that maintains state throughout the solving process. It stores both the learned density patterns from training and the evolving solution as new deliveries are routed.

**Key Attributes:**
- `model`: Fitted KMeans clusterer for macro-level partitioning
- `density_info`: Dictionary storing KDE contours, DBSCAN models, and ordered centroids for each cluster
- `subroutes`: Hierarchical dictionary organizing deliveries by cluster → contour level → DBSCAN label
- `vehicles`: List of completed vehicle routes
- `n_clusters`: Number of KMeans clusters (hyperparameter)
- `eps`: DBSCAN epsilon parameter controlling density threshold (hyperparameter)
- `min_samples`: DBSCAN minimum samples for core points (hyperparameter)
- `is_countor`: Boolean flag enabling/disabling contour-based density separation

### Method 1: `pretrain()` - Learning Density Patterns from Historical Data

**Purpose**: This method implements the core research contribution - a hierarchical density-based clustering approach that learns spatial patterns from training instances.

**Density Estimation Workflow:**

1. **Aggregate Training Data**: Collect all delivery points from multiple historical instances
2. **Macro-Clustering**: Apply KMeans to partition space into `n_clusters` regions
3. **Density Analysis (KDE)**: For each cluster:
   - Compute Kernel Density Estimation over the cluster points
   - Generate contour levels representing density gradients
   - Identify high-density zones (≥4 nested contours)
4. **Density-Based Separation**: Classify points into:
   - **High-density points**: Points within ≥4 contours (urban cores, dense areas)
   - **Low-density points**: Points outside high-density contours
5. **Micro-Clustering (DBSCAN)**: Apply DBSCAN only to low-density points, creating sub-clusters that respect natural spatial groupings
6. **Centroid Ordering**: Compute centroids for each DBSCAN cluster and sequence them using nearest-neighbor ordering

**Research Insight**: By separating high-density from low-density points before DBSCAN, we prevent over-fragmentation in dense urban areas while maintaining fine-grained clustering in sparse regions. This adaptive approach better matches real-world delivery patterns.

**Output**: 
- `density_info`: Dictionary mapping each KMeans cluster to its KDE contours, DBSCAN model, and ordered centroids
- `train_map`: Interactive Folium visualization showing all clustering levels

### Method 2: `route()` - Density-Aware Delivery Assignment

**Purpose**: Assign each new delivery to the appropriate hierarchical cluster based on learned density patterns.

**Clustering Assignment Logic:**

1. **Macro-Assignment**: Use trained KMeans model to predict which main cluster the delivery belongs to
2. **Density Classification**: Evaluate the delivery's position relative to KDE contours:
   - Check how many contour levels contain the point
   - High-density: ≥4 contours → Special handling (contour=-1, label=-99)
   - Low-density: <4 contours → Apply DBSCAN prediction
3. **Micro-Assignment**: For low-density points, use `dbscan_predict()` to find nearest DBSCAN sub-cluster
4. **Sub-route Placement**: Build hierarchical key `k_{cluster}_contour_{level}_label_{dbscan_label}` and add delivery to corresponding sub-route
5. **Capacity Check**: If sub-route exceeds vehicle capacity, finalize current route and start new vehicle

**Research Insight**: The conditional logic based on density level (`is_countor` flag) allows testing whether density-aware separation improves routing quality compared to uniform clustering.

### Method 3: `finish()` - Consolidating Sub-routes into Vehicle Routes

**Purpose**: Merge remaining deliveries from all sub-routes into complete vehicle routes, respecting capacity constraints.

**Consolidation Process:**

1. **Ordered Processing**: Iterate through sub-routes in the spatial order determined by centroid sequencing
2. **Incremental Packing**: Use a greedy bin-packing approach:
   - Maintain a `base_subroute` buffer for the current vehicle
   - Add deliveries one-by-one while capacity permits
   - When capacity exceeded, finalize current vehicle and start new one
3. **TSP Optimization**: Each finalized vehicle route is optimized using OR-Tools TSP solver
4. **Final Cleanup**: Process any remaining deliveries in the buffer

**Research Consideration**: The centroid-based ordering ensures that spatially proximate sub-routes are processed consecutively, promoting spatial coherence in the final routes and reducing inter-route distances.

In [16]:
@dataclass
class KMeansSolver:
    model: Optional[KMeans] = None
    subroutes: Optional[Dict[int, List[int]]] = None
    vehicles: Optional[List[int]] = None
    vehicle_capacity: Optional[int] = None
    origin: Optional[Point] = None
    n_clusters: int = None
    eps = 0.01
    min_samples = 20
    density_info: dict = field(default_factory=dict)
    train_map = None
    osrm_config: OSRMConfig = field(default_factory=lambda: OSRMConfig(host="http://localhost:5001"))

    def __init__ (self, is_countor, n_clusters, eps, min_samples):
        self.is_countor = is_countor
        self.n_clusters = n_clusters
        self.eps = eps
        self.min_samples = min_samples
        # Initialize default factory fields manually since we override __init__
        self.density_info = {}
        self.osrm_config = OSRMConfig(host="http://localhost:5001")
        

    def pretrain(self, train_instances):
        """
        Learn spatial density patterns from historical delivery instances.
        Implements hierarchical clustering: KMeans → KDE → DBSCAN.
        """
        # Prepare color map for displaying contours on the training map
        cm = px.colors.sequential.Plasma
        
        # Aggregate delivery points from all training instances
        all_points_list = []
        for instance in train_instances:
            all_points_list.extend(_get_delivery_coordinates(instance))

        # Initialize training visualization map centered on first point
        points = np.array(all_points_list)

        # STAGE 1: Macro-level clustering with KMeans
        # Prepara o mapa de treino
        m = folium.Map(
            location=(points[0][0], points[0][1]),
            zoom_start=12,
        )

        # Add cluster centroids to visualization map
        # Constrói o modelo com KMeans
        self.model = KMeans(n_clusters=self.n_clusters).fit(points)

        # Exibe os clusters no mapa
        markers = add_clusters_to_markers(self.model)
        for marker in markers:
            folium.Circle([marker['lat'], marker['lng']], color=marker['color'], radius=5000, weight=3, popup="5 km").add_to(m)
        
        # Initialize sub-route dictionary for hierarchical organization
        subroutes = {}
        feature_groups = dict()
        
        predictions = self.model.predict(points)

        
        # Process each KMeans cluster independently
        for i in range(self.n_clusters):

            # Extract points belonging to cluster i
            points_cluster_i = list(compress(points, predictions == i))

            # STAGE 2: Density estimation with KDE for cluster i
            x, y, xx, yy, f = set_contours(points_cluster_i)
            contours, contour_plot = plot_contours (xx = xx, yy = yy, f = f, x = x, y = y)

            # Classify points by density level (high vs low density)
            points_cluster_i_and_external_contours = []
            centroids = dict()
            for pos in range(len(points_cluster_i)):
    
                # Determine contour level for this point (higher = denser)
                inside = find_contours([points_cluster_i[pos][0]], [points_cluster_i[pos][1]], contours)
                point_contour = int(inside[0]) if len(inside) > 0 else 0

                # Visualize high-density points with color-coded markers
                if point_contour >= 4:
                    folium.CircleMarker(
                        [points_cluster_i[pos][0], points_cluster_i[pos][1]], color=cm[point_contour], radius=1, weight=1
                    ).add_to(m)

                    # Group high-density points into 'origin' centroid category
                    if 'origin' in centroids.keys():
                        centroids['origin'] = centroids['origin'] + [points_cluster_i[pos]]
                    else:
                        centroids['origin'] = [points_cluster_i[pos]]
        
                # Low-density points: will be clustered with DBSCAN
                if not (point_contour >= 4):
                    points_cluster_i_and_external_contours += [points_cluster_i[pos]]

            # Fallback: if all points are high-density, use them for DBSCAN anyway
            if 'origin' not in centroids.keys():
                centroids['origin'] = points_cluster_i_and_external_contours

            # STAGE 3: Micro-level clustering with DBSCAN on low-density points
            db = DBSCAN(eps=self.eps, min_samples=self.min_samples).fit(points_cluster_i_and_external_contours)
            db_predictions = db.labels_

            # Prepare subroutes
            subroutes_cluster, labels_cluster = prepare_sub_routes(db_predictions, i)
            subroutes = subroutes | subroutes_cluster

            # Change the name of the labels, according to cluster i
            labels = ["cluster_{}_{}".format(i, item) for item in labels_cluster]
            db_predictions = ["cluster_{}_{}".format(i, item) for item in db_predictions]

            # Adiciona um feature group para cada label
            for l in labels:
                feature_groups[l] = folium.FeatureGroup(name=l).add_to(m)

            # Exibe os pontos fora dos inners contours no mapa
            for pos_marker, m_marker in enumerate(points_cluster_i_and_external_contours):
                folium_marker = folium.CircleMarker(location=[m_marker[0], m_marker[1]], radius=1)
                folium_marker.add_to(feature_groups[db_predictions[pos_marker]])

            # Calcula os pontos médios de cada cluster
            #centroids = dict()
            for pos, pred in enumerate (db_predictions):
                if pred in centroids.keys():
                    centroids[pred] = centroids[pred] + [points_cluster_i_and_external_contours[pos]]
                else:
                    centroids[pred] = [points_cluster_i_and_external_contours[pos]]

            for k, v in centroids.items():
                centroids[k] = [
                    pd.DataFrame(v, columns = ['lat', 'lng'])[['lat']].values.mean(),
                    pd.DataFrame(v, columns = ['lat', 'lng'])[['lng']].values.mean()
                ]
            
            # Sequencia centroids
            formatted_centroids = []
            origin  = train_instances[0].origin

            for k, v in centroids.items():
                
                # Cria um objeto Point para cada centroid
                if k == 'origin':
                    origin = Point (lat=v[0], lng=v[1])
                if k == "cluster_{}_{}".format(i, "-1"):
                    pass
                else:

                    p = Point (
                        lat=v[0], 
                        lng=v[1]
                    )
                    d = Delivery(
                        #id = str(uuid.uuid4()), 
                        id = k,
                        point = p, 
                        size = 0
                    )
                    formatted_centroids.append(d)

            ordered_centroids = self._construct_vehicle(formatted_centroids, origin)

            # remove a origin do ordered_centroids
            ordered_centroids = reorder_deliveries_by_distance(ordered_centroids.deliveries)
            ordered_centroids = [d for d in ordered_centroids if d.id != 'origin']
            

            # Exibe os clusters densos no mapa
            for k, v in centroids.items():
                
                # Cria um objeto Point para cada centroid
                if k == 'origin':
                    folium_marker = folium.Marker(location=[v[0], v[1]], radius=10, tooltip="Origin", icon=folium.Icon(color='black'))
                    folium_marker.add_to(m)
                else:
                    found = False
                    for pos_ordered_centroids, c in enumerate(ordered_centroids):
                        if v[0] == c.point.lat and v[1] == c.point.lng:
                            folium_marker = folium.Marker(location=[v[0], v[1]], radius=10, tooltip="Cluster name: {} | Sequence: {}".format(k, pos_ordered_centroids), icon=folium.Icon(color='red'))
                            folium_marker.add_to(feature_groups[k])
                            found = True

                    if not found:
                        folium_marker = folium.Marker(location=[v[0], v[1]], radius=10, tooltip="Cluster name: {} | Sequence: {}".format(k, "not found"), icon=folium.Icon(color='red'))
                        folium_marker.add_to(feature_groups[k])
        
            # Number of clusters in labels, ignoring noise if present.
            n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
            n_noise_ = list(labels).count(-1)
            
            print("Estimated number of clusters: %d" % n_clusters_)
            print("Estimated number of noise points: %d" % n_noise_)
            
            # Add density_info
            self.density_info[i] = {
                'x': x,
                'y': y,
                'xx': xx,
                'yy': yy,
                'f': f,
                'contours': contours,
                'contour_plot': contour_plot,
                'db': db,
                'ordered_centroids': ordered_centroids,
                'subroutes_cluster': subroutes_cluster
            }

        # Adiciona legenda ao mapa
        m.add_child(folium.LayerControl(position='topright', collapsed=False, autoZIndex=True))
        
        # Inicializa a variável com os veículos completos
        vehicles = []

        # Ordena subroutes baseado em ordered_centroids
        ordered_subroutes = align_subroutes(self.density_info)

        self.subroutes = subroutes
        self.ordered_subroutes = ordered_subroutes
        self.vehicles = vehicles

        # Armazena propriedades do problema
        self.vehicle_capacity = train_instances[0].vehicle_capacity
        self.origin = train_instances[0].origin
        self.name = train_instances[0].name        
        self.train_map = m 

    def finetune(self, delivery):
        """Nada é feito aqui, logo, deixe o código em branco"""
        pass

    def route(self, delivery):
        delivery_point = np.array([(delivery.point.lat, delivery.point.lng)])
        subregion_index = self.model.predict(delivery_point)[0]
        label_index = dbscan_predict (self.density_info[subregion_index], delivery_point)[0]

        # Calcula o contour
        inside = find_contours([delivery.point.lat], [delivery.point.lng], self.density_info[subregion_index]['contours'])
        point_contour = int(inside[0]) if len(inside) > 0 else 0
   
        # Contexto da solução
        if self.is_countor:
            if (point_contour >= 4):
                point_contour = -1
                label_index = -99
            else:
                point_contour = 0
        else:
            point_contour = 0
            label_index = -99

        #position = str(subregion_index) + str(point_contour) + str(label_index)
        position = "k_" + str(subregion_index) + '_contour_' + str(point_contour) + "_label_" + str(label_index)
        
        # Verifica se o veículo em `subregion_index` comporta o novo pacote
        if (
            _compute_vehicle_volume(self.subroutes[position])
            + delivery.size
            <= self.vehicle_capacity
        ):
            self.subroutes[position].append(delivery)
        else:
            # Finaliza a rota atual
            
            #print("route - fechando subroute {}".format(position))
            
            vehicle_solution = self._construct_vehicle(
                self.subroutes[position]
            )
            self.vehicles.append(vehicle_solution)

            # Adiciona um novo veículo a esta sub-região
            self.subroutes[position] = [delivery]
    
    def _construct_vehicle(self, vehicle_deliveries = None, origin = None):
        distance_matrix = self._compute_distance_matrix(vehicle_deliveries, origin)
        ordered_indices, _ = solve_tsp_ortools(distance_matrix)

        # `ordered_indices` tem o formato `[0, 4, 3, 1, ..., 0]`. Precisamos
        # remover a origem do início e do final e ordenar a variável
        # `vehicle_deliveries` usando estes índices
        ordered_vehicle_deliveries = []
        for ordered_index in ordered_indices[1:-1]:
            ordered_vehicle_deliveries.append(
                vehicle_deliveries[ordered_index - 1]
            )

        return CVRPSolutionVehicle(
            origin=self.origin, deliveries=ordered_vehicle_deliveries
        )

    def _compute_distance_matrix(self, vehicle_deliveries = None, origin = None):
        # Os pontos do problema consistem na origem mais as entregas em
        # `vehicle_deliveries`
        if origin:
            points = [origin]
        else:
            points = [self.origin]
            
        for delivery in vehicle_deliveries:
            points.append(delivery.point)

        # Configuração com o servidor para os alunos
        return calculate_distance_matrix_m(points, config=self.osrm_config)

    def finish(self):
        
        # Faz o merge de subroutes não finalizadas (buffer)
        #subroute_keys = self.subroutes.keys()
        base_subroute = []
        
        for k, v in self.ordered_subroutes.items():

            #print("Finish pos {}".format(k))
            key = list(v.keys())[0]

            if key not in self.subroutes.keys():
                continue
            
            subroute = self.subroutes[key]

        #for key, subroute in self.subroutes.items():
        #    print("Processing {}".format(key))
            
            for delivery in subroute:
            
                if (
                    _compute_vehicle_volume(base_subroute)
                    + delivery.size
                    <= self.vehicle_capacity
                ):
                    base_subroute.append(delivery)
                    
                else:
                    # Finaliza a rota atual
                    
                    #print("custom - fechando subroute")
                    
                    vehicle_solution = self._construct_vehicle(
                        base_subroute
                    )
                    self.vehicles.append(vehicle_solution)
        
                    # Adiciona um novo veículo a esta sub-região
                    base_subroute = [delivery]

            
        if base_subroute != []:
            #print("finish - fechando base route")
            vehicle_solution = self._construct_vehicle(base_subroute)
            self.vehicles.append(vehicle_solution)
                
        """
        # Para cada sub-rota sobrando em `subroutes`, construa um novo veículo
        #for subroute in self.subroutes.values():
        for key, subroute in self.subroutes.items():
            
            if subroute != []:
                print("finish - fechando subroute {}".format(key))
                vehicle_solution = self._construct_vehicle(subroute)
                self.vehicles.append(vehicle_solution)
        """
        
        # Ao final, retorne uma variável do tipo `CVRPSolution`
        return CVRPSolution(name=self.name, vehicles=self.vehicles)

## Solver Execution Function

This section implements the main orchestration function that coordinates the entire solving process. The `solve()` function serves as the entry point for applying the hybrid density-based clustering approach to CVRP instances.

### Workflow Overview

1. **Initialization**: Create a KMeansSolver with specified hyperparameters
2. **Training**: Learn density patterns from historical instances via `pretrain()`
3. **Validation**: Verify that all clusters have valid centroid orderings
4. **Routing**: Process each delivery through `finetune()` and `route()` methods
5. **Finalization**: Consolidate all routes into a complete CVRP solution

### Hyperparameters

- **is_countor**: Boolean flag enabling density-based point separation (research ablation parameter)
- **n_clusters**: Number of KMeans macro-regions (spatial partitioning granularity)
- **eps**: DBSCAN epsilon - maximum distance for neighborhood inclusion (density sensitivity)
- **min_samples**: DBSCAN minimum points to form a dense core (cluster formation threshold)

In [17]:
def solve(train_instances, problem, is_countor, n_clusters, eps, min_samples):
    """
    Execute the complete CVRP solving process using hybrid density-based clustering.
    
    Args:
        train_instances: List of CVRPInstance objects for training density patterns
        problem: CVRPInstance to solve
        is_countor: Boolean - if True, use contour-based density separation
        n_clusters: Number of KMeans clusters for macro-level partitioning
        eps: DBSCAN epsilon parameter for density neighborhood
        min_samples: DBSCAN minimum samples for core point classification
        
    Returns:
        tuple: (CVRPSolution, KMeansSolver) or (False, False) if solving fails
    """
    # Initialize solver with specified hyperparameters
    solver = KMeansSolver(
        is_countor=is_countor, 
        n_clusters=n_clusters, 
        eps=eps, 
        min_samples=min_samples
    )

    # Train the solver on historical delivery instances
    solver.pretrain(train_instances)

    print("Training completed")

    # Validate that all clusters have valid centroid orderings
    for k, v in solver.density_info.items():
        if len(v['ordered_centroids']) == 0:
            # Return failure if any cluster has no centroids
            return False, False

    # Process each delivery in the problem instance
    for delivery in problem.deliveries:
        solver.finetune(delivery)  # Currently a no-op, reserved for future use
        solver.route(delivery)     # Assign delivery to appropriate cluster/sub-cluster

    # Finalize and return the complete solution
    return solver.finish(), solver

## Single Instance Testing

This section demonstrates the solver on a single CVRP instance, comparing two approaches:

1. **Default Mode** (`is_countor=False`): All low-density points treated uniformly
2. **Contour Mode** (`is_countor=True`): High-density points (≥4 contours) separated from low-density points

This comparison allows evaluation of the density-aware separation strategy's impact on solution quality.

### Load Training and Test Instances

Load historical instances from the training set (pa-0 region) and a specific test instance for evaluation.

In [ ]:
def _load_instances(train_path_str):
    """
    Load all CVRP instances from a directory.
    
    Args:
        train_path_str: Path to directory containing CVRP instance JSON files
        
    Returns:
        list: List of CVRPInstance objects
    """
    # Convert directory path string to Path object
    train_path = Path(train_path_str)

    instances = []
    for instance_file in train_path.iterdir():
        instance = CVRPInstance.from_file(instance_file)
        instances.append(instance)

    return instances

# Load training instances from the pa-0 region
train_path_str = "../data/cvrp-instances-1.0/train/pa-0"
train_instances = _load_instances(train_path_str)

# Load the test problem instance (pa-90)
# Alternative instance for testing: cvrp-0-pa-101.json
problem = CVRPInstance.from_file("../data/cvrp-instances-1.0/dev/pa-0/cvrp-0-pa-90.json")

# Solve using default approach (no density separation)
print("Solving with default mode (is_countor=False)")
solution_default, solver_default = solve(train_instances, problem, False, 2, 0.01, 10)

# Solve using contour-based density separation
print("Solving with contour mode (is_countor=True)")
solution_countor, solver_countor = solve(train_instances, problem, True, 2, 0.01, 10)

Resolving default
Estimated number of clusters: 31
Estimated number of noise points: 0
Estimated number of clusters: 31
Estimated number of noise points: 0
Estimated number of clusters: 33
Estimated number of noise points: 0
Training completed
Estimated number of clusters: 33
Estimated number of noise points: 0
Training completed
Resolving contour
Resolving contour
Estimated number of clusters: 31
Estimated number of noise points: 0
Estimated number of clusters: 31
Estimated number of noise points: 0
Estimated number of clusters: 33
Estimated number of noise points: 0
Training completed
Estimated number of clusters: 33
Estimated number of noise points: 0
Training completed


### Save Training Map Visualization

Export the training density map showing KMeans clusters, KDE contours, and DBSCAN sub-clusters as an interactive HTML file.

In [ ]:
solver_countor.train_map.save('delivery_map.html')

### Evaluate and Compare Solutions

Calculate the total distance for both solutions and compute the improvement percentage achieved by the contour-based approach.

In [ ]:
result_default = evaluate_solution(problem, solution_default, config=osrm_config)
result_countor = evaluate_solution(problem, solution_countor, config=osrm_config)

print('Default solution total distance: {}'.format(result_default))
print('Contour solution total distance: {}'.format(result_countor))
print('Improvement: {:.2%}'.format((result_default/result_countor)-1))

### Visualize Default Solution

Plot the vehicle routes generated by the default solver (without density-based separation).

In [ ]:
plot_cvrp_solution(solution_default)

### Visualize Contour-Based Solution

Plot the vehicle routes generated by the contour-based solver (with density-aware separation).

In [ ]:
plot_cvrp_solution(solution_countor)

### Visualize Problem Instance

Display the original problem instance showing all delivery points and the depot location.

In [ ]:
plot_cvrp_instance(problem)

## Hyperparameter Grid Search Experiment

This section conducts a comprehensive hyperparameter search across multiple CVRP instances to evaluate the density-aware clustering approach. The experiment systematically tests different combinations of clustering parameters to identify optimal configurations.

### Experimental Design

**Test Instances**: 30 problem instances (pa-90 through pa-119) from the dev set

**Hyperparameter Grid**:
- **n_clusters**: 1, 2, 3, 4, 5 (KMeans macro-regions)
- **eps**: 0.01, 0.05, 0.1, 0.5 (DBSCAN density threshold)
- **min_samples**: 10, 20, 30, 40 (DBSCAN core point requirement)

**Total Configurations**: 5 × 4 × 4 = 80 hyperparameter combinations per instance

### Evaluation Methodology

For each configuration:
1. Solve with **default mode** (is_countor=False) - uniform density treatment
2. Solve with **contour mode** (is_countor=True) - density-aware separation
3. Compare solutions using OSRM-based total distance
4. Calculate improvement percentage: `(default_distance / contour_distance) - 1`

### Research Question

Does density-aware point separation (contour mode) consistently improve solution quality across different hyperparameter settings and problem instances?

### Execute Grid Search

Run nested loops over instances and hyperparameters, collecting improvement metrics for each configuration.

In [ ]:
# Initialize results dictionary to store improvement metrics
res = dict()

# Loop over 30 test instances (pa-90 through pa-119)
for i in range(90, 120):

    # Loop over number of KMeans clusters
    for n in range(1, 6):

        # Loop over DBSCAN epsilon values (density neighborhood radius)
        for eps in [0.01, 0.05, 0.1, 0.5]:

            # Loop over DBSCAN min_samples values (core point threshold)
            for min_samples in [10, 20, 30, 40]:

                # Create unique key for this hyperparameter configuration
                key = "N_{}_EPS_{}_MIN_SAMPLES_{}".format(n, eps, min_samples)
                print(key)
                
                # Construct filename for current instance
                filename = "../data/cvrp-instances-1.0/dev/pa-0/cvrp-0-pa-{}.json".format(i)
                print(filename)
                
                # Load the problem instance
                problem = CVRPInstance.from_file(filename)
                
                # Solve with default mode (no density separation)
                print("Solving with default mode")
                solution_default, solver_default = solve(
                    train_instances, problem, False, n, eps, min_samples
                )
                
                # Solve with contour mode (density-aware separation)
                print("Solving with contour mode")
                solution_countor, solver_countor = solve(
                    train_instances, problem, True, n, eps, min_samples
                )

                # Skip if either solver failed to produce a solution
                if not solver_default or not solver_countor:
                    print('Could not find a complete solution for n_clusters {}'.format(n))
                    continue
            
                # Evaluate both solutions using OSRM distances
                result_default = evaluate_solution(problem, solution_default, config=osrm_config)
                result_countor = evaluate_solution(problem, solution_countor, config=osrm_config)
                
                # Display results for this configuration
                print('Default solution distance: {}'.format(result_default))
                print('Contour solution distance: {}'.format(result_countor))
                print('Improvement: {:.2%}'.format((result_default/result_countor)-1))
                
                # Store improvement metric for this configuration
                # Accumulate results across multiple instances
                try:
                    res[key] = res[key] + [(result_default/result_countor)-1]
                except:
                    res[key] = [(result_default/result_countor)-1]

### Display Average Improvements

Calculate and display the mean improvement percentage for each hyperparameter configuration across all test instances.

In [ ]:
# Calculate and display mean improvement for each configuration
for k, v in res.items():
    # Convert improvement ratios to percentages and compute mean
    r = np.mean([i * 100 for i in v])
    print("Configuration {} ==> Average improvement: {:.2f}%".format(k, r))

### Aggregate and Export Results

Process the raw results to identify the best hyperparameter configuration for each n_clusters value. This analysis helps determine optimal settings for the density-aware clustering approach.

In [ ]:
# Convert results dictionary to DataFrame for analysis
export = pd.DataFrame(data=res).T

# Rename the first column to 'metric' for clarity
export = export.rename(columns={0: 'metric'})

# Parse hyperparameter values from configuration keys
# Split keys like "N_2_EPS_0.01_MIN_SAMPLES_10" into separate columns
export.index = export.index.str.split('_')
export[['N', 'N_Value', 'EPS', 'EPS_Value', 'MIN', 'MIN_SAMPLES', 'SAMPLES_Value']] = pd.DataFrame(
    export.index.tolist(), index=export.index
)
export.reset_index(drop=True, inplace=True)

# Define aggregation strategy: keep maximum improvement metric
aggregations = dict()
aggregations['metric'] = 'max'

# First aggregation: find best configuration for each (N, EPS, MIN_SAMPLES) combination
export = export.groupby(['N_Value', 'EPS_Value', 'SAMPLES_Value'], as_index=False).agg(aggregations)

# Second aggregation: find best overall configuration for each N_Value
export = export.groupby(['N_Value'], as_index=False).agg(aggregations)

# Export results to CSV for further analysis
export.to_csv('export.csv')

# Deduplicate by keeping the row with maximum metric for each N_Value
export = export.loc[export.groupby('N_Value')['metric'].idxmax()]

# Reset index and display final results
export.reset_index(drop=True, inplace=True)
export